# Querying an API

In this lab, we will practice following the ETL (extract, transform, load) pattern to pull data from an external API, manipulate the data so we can use it, and finally make Python objects with the data.  We will walk through obtaining JSON from an API and converting that JSON into objects we can use in our applications.  All APIs aren't exactly the same, but we will understand the general flow for accessing API data by the end of this lab.

## Objectives

* Understand how to create a personal key to gain access to an API's data
* Understand how to clean the data and create Python objects from it

## Beatles Concerts

We will be working with the [setlist.fm](https://api.setlist.fm/docs/1.0/index.html) API to obtain information about Beatles concerts from 1965!  First, we'll need to be authorized to access this data before we can worry about setting up our models and relationships that we'll use later on to make related objects.  

#### Part 1: Extraction

As with most APIs, setlist.fm requires us to create an account and apply for our own API key to access its data.  Go to the link above and click on "register first" to fill out a form for creating an account.  Check your email for a message asking to confirm the newly created account.

Once confirmation is complete, we can go back to the link above to "apply for an API key".  Fill out this form and we'll be assigned our very own, unique access token.

Sweet! Now let's test our API key.  Click on the "Try it out!" button underneath the Resources header.  Now we can add our API key to the input field at the top part of the page.

Let's try getting data on Beatles concerts from the 1965.  Under `/1.0/search/setlists`, set the search parameters for `artistName` to "The Beatles", `p` to 1, and `year` to 1965, respectively.  Also, we want our response data to be in the JSON format, so let's set the "Response Content Type" dropdown to "application/json".  Now, click on "Try it out!"

![test_response](test_response.png)

Awesome!  Our request seems to be working properly.

**`setlist_data.py`**

Now that we know that our API key is working, we will make our request in the setlist_data.py file so that our program can access the API's data.  We need to import requests at the top of the file.  Just as we did in our test run above, we will want to make the same GET request with headers for accepting a JSON response and our API key.  We can pass these into our GET request by means of a dictionary as we can see below:

```python
token = 'your-token-goes-here'
headers = {'Accept': 'application/json', 'x-api-key': token}

response = requests.get('https://api.setlist.fm/rest/1.0/search/setlists?artistName=The%20Beatles&p=1&year=1965', headers=headers)
raw_data = json.loads(response.content)
```

Uncomment the debugger in the middle of the file to play around with out response data.  If we look at `response.content`, we see that the content of response variable above is one long JSON string.  We want to work with the dictionary inside that string, which is why we need to import json at the top of the file and use `json.loads` to transform our response's content into a format we can work with later.

#### Part 2: Transform

Take another look at the `raw_data` -- it is one long dictionary. However, we are interested in the setlist data for each concert.  How can we pull only the setlist information from this object?  We want our `data` to be a list of the dictionaries, with each dictionary representing the data for one concert.